In [46]:
import pandas as pd
import numpy as np

In [277]:
ibc_df = pd.read_csv("incident-based_crime_08-12.csv", skiprows=1, skipfooter=35, engine="python")

# Clean Incident Based Crime

ibc_df["Geography"] = ibc_df["Geography"].str.replace(r"\s\(\d+(?:,\d+)*\)", '', regex=True)
ibc_df["Violations"] = ibc_df["Violations"].str.replace(r"\s\(\d+(?:,\d+)*\)", '', regex=True)
# ibc_df.head()
# ibc_df.tail()

In [279]:
provinces = ["Alberta", "British Columbia", "Manitoba", "New Brunswick", "Newfoundland and Labrador", "Nova Scotia", "Ontario", "Prince Edward Island", "Quebec", "Saskatchewan"]

provinces_crimes_df = ibc_df[ibc_df["Geography"].str.fullmatch("|".join(provinces))]

# Pivoted df contains the value Rate per 100k population of the differnt violations in 2008
pivoted_df = provinces_crimes_df.pivot_table(index="Violations", columns="Geography", values="2008").reset_index()

In [ ]:
cities = ["Toronto", "Brampton", "Montreal", "Halifax", "Calgary", "Fredericton", "Vancouver"]
crimes = ["Homicide", "Murder, first degree", "Attempted murder", "Sexual assault, level 3 aggravated", "Sexual exploitation", "Assault, level 3, aggravated", "Robbery", "Kidnapping", "Extortion", "Incest", "Trafficking in persons", "Fraud", "Shoplifting over $5,000", "Arson", "Total prostitution", "Total impaired driving", "Total Drug violations"]

# Pivoted df contains the value Rate per 100k population of the differnt violations in 2008
pivoted_df = pivoted_df[pivoted_df["Violations"].str.contains("|".join(crimes), regex=True)].reset_index(drop=True)
pivoted_df

Geography,Violations,Alberta,British Columbia,Manitoba,New Brunswick,Newfoundland and Labrador,Nova Scotia,Ontario,Prince Edward Island,Quebec,Saskatchewan
0,Arson,52.39,54.63,69.09,61.32,32.78,45.23,20.17,27.95,48.31,75.07
1,"Assault, level 3, aggravated",18.76,9.65,27.12,7.77,5.73,9.07,10.08,1.43,3.32,35.91
2,Attempted murder,2.12,2.30,2.99,0.94,0.39,3.20,2.13,0.72,1.97,3.16
3,Extortion,3.17,6.16,1.16,1.87,0.59,3.31,3.71,2.87,5.83,1.58
4,Fraud,350.96,321.76,206.77,256.26,224.35,293.98,279.88,253.68,209.69,316.83
5,Homicide,3.06,2.67,4.48,0.40,0.99,1.28,1.36,1.43,1.19,2.96
6,Total impaired driving,475.34,326.25,251.06,364.03,312.63,284.49,145.00,333.22,211.31,605.95
7,Total prostitution,19.04,19.27,15.43,8.84,2.76,10.13,8.27,0.00,8.41,20.12


In [273]:
"""
Idea is for analyzing crimes in different cities, use total violations
Then for trends over time use the incident base crime from canada
for analyzing and identifying regions with higher or lower crime rates 
For analyzig crime rate use line charts with different colours the show different crimes, general canada crimes, use 2015 data
For comparing crime rates with socioeconomic factors use 2016 census data of cities with the analyzed crime rate
"""

'\nIdea is for analyzing crimes in different cities, use total violations\nThen for trends over time use the incident base crime from canada\nfor analyzing and identifying regions with higher or lower crime rates \nFor analyzig crime rate use line charts with different colours the show different crimes, general canada crimes, use 2015 data\nFor comparing crime rates with socioeconomic factors use 2016 census data of cities with the analyzed crime rate\n'

In [ ]:
canada_crime_rate = ibc_df[ibc_df["Geography"].str.fullmatch("Canada")]
canada_crime_rate = canada_crime_rate[canada_crime_rate["Violations"].str.contains("|".join(crimes), regex=True)]
canada_crime_rate.reset_index(inplace=True)
canada_crime_rate.drop(columns=["Geography", "index"], inplace=True)
canada_crime_rate.set_index("Violations", inplace=True)

# canada_crime_rate contains the value Rate per 100k population of the differnt violations in 2008, 2009, 2010, 2011, 2012
canada_crime_rate

,2008,2009,2010,2011,2012
Violations,,,,,
Homicide,1.83,1.81,1.62,1.73,1.56
Attempted murder,2.16,2.37,1.96,1.93,1.94
"Assault, level 3, aggravated",10.78,10.73,10.20,10.23,10.07
Extortion,4.21,5.09,4.62,4.43,4.91
Fraud,274.22,267.34,239.58,225.44,224.86
Arson,40.13,39.74,35.85,30.16,31.65
Total prostitution,11.47,10.48,8.85,7.11,5.95
Total impaired driving,254.20,261.82,255.61,259.85,242.21


In [56]:
# import chardet
# with open("Fredericton_Census_Data_2016.csv", "rb") as f:
#     result = chardet.detect(f.read())
#     print(result)  # Check the detected encoding

In [361]:

cities_census_dfs = [pd.read_csv(f"CensusData/{city}_Census_Data_2016.csv", skiprows=[0, 2], skipfooter=540, encoding="ISO-8859-1", engine="python") for city in cities]

census_data = ["Population; 2016", "Median total income in 2015 among recipients ($)", "Average total income in 2015 among recipients ($)", "Immigrants", "Non-immigrants"]

# immigrant_country = ["Asia", "Africa", "Europe"]
# To make analysis easier you could remove this part, (comment out the part that filters out Immigrants by selected place of birth)
immigrant_country = ["Brazil", "United Kingdom", "Nigeria", "India", "China"]
socio_economic_dfs = [
    df[
        (df["Characteristics"] == "Population; 2016") |
        (df["Characteristics"].str.strip() == "Median total income in 2015 among recipients ($)") |
        (df["Characteristics"].str.strip() == "Average total income in 2015 among recipients ($)") |
        ((df["Topic"].str.strip() == "Immigrant status and period of immigration") & (df["Characteristics"].str.strip() == "Immigrants")) |
        ((df["Topic"].str.strip() == "Immigrants by selected place of birth") & (df["Characteristics"].str.strip().str.fullmatch("|".join(immigrant_country)))) |
        ((df["Topic"].str.strip() == "Admission category and applicant type") & (df["Characteristics"].str.strip() == "Refugees")) |
        ((df["Topic"].str.strip() == "Highest certificate; diploma or degree") & (df["Characteristics"].str.strip() == "No certificate; diploma or degree")) |
        # ((df["Topic"].str.strip() == "Major field of study - Classification of Instructional Programs (CIP) 2016") & (df["Characteristics"].str.strip() == "Mathematics; computer and information sciences")) |
        ((df["Topic"].str.strip() == "Labour force status") & (df["Characteristics"].str.strip() == "Unemployed"))
    ]
    for df in cities_census_dfs
]

cities_census_dfs[0].head(10)
for index, df in enumerate(socio_economic_dfs):
    socio_economic_dfs[index] = df.loc[:, ["Characteristics", "Total", "Male", "Female"]].reset_index().drop(columns="index").drop_duplicates(subset=["Characteristics"], keep="last")

socio_economic_dfs[0]

characteristics = {
    "Population; 2016": "Population", "Median total income in 2015 among recipients ($)": "Median total income", "Average total income in 2015 among recipients ($)": "Average total income", 
    "Brazil": "Brazil Immigrants", "United Kingdom": "United Kingdom Immigrants", "Nigeria": "Nigeria Immigrants", "China": "China Immigrants", "India": "India Immigrants"}

for index, df in enumerate(socio_economic_dfs):
    total_population = df.at[0, "Total"]
    socio_economic_dfs[index]["Rate per 100k population"] = df["Total"]
    socio_economic_dfs[index].loc[3:, "Rate per 100k population"] = df.iloc[3:,:]["Total"].apply(lambda x: (x/total_population)*100000)
    socio_economic_dfs[index] = socio_economic_dfs[index].drop(0).reset_index(drop=True)

    for key, value in characteristics.items():
            socio_economic_dfs[index].loc[socio_economic_dfs[index]["Characteristics"].str.strip() == key, "Characteristics"] = value

In [362]:
merge_prep_dfs = [df.loc[:, ["Characteristics", "Rate per 100k population"]].rename(columns={"Rate per 100k population": f"{cities[index]} Rate per 100k population"}) for index, df in enumerate(socio_economic_dfs)]

merged_socio_economic_df = merge_prep_dfs[0]

for df in merge_prep_dfs[1:]:
    merged_socio_economic_df = pd.merge(merged_socio_economic_df, df, how="left", on="Characteristics")


merged_socio_economic_df

,Characteristics,Toronto Rate per 100k population,Brampton Rate per 100k population,Montreal Rate per 100k population,Halifax Rate per 100k population,Calgary Rate per 100k population,Fredericton Rate per 100k population,Vancouver Rate per 100k population
0,Median total income,30089.000000,29092.000000,28321.000000,36098.000000,43333.000000,33615.000000,32964.000000
1,Average total income,52268.000000,37647.000000,40079.000000,46440.000000,67741.000000,43116.000000,50317.000000
2,Immigrants,46347.138698,52016.548806,33492.227931,9227.769633,30911.783218,11070.079011,41610.582024
3,Brazil Immigrants,257.178012,65.696603,207.075287,33.487874,114.588209,17.176228,163.107337
4,United Kingdom Immigrants,1475.158434,1461.328284,241.392297,1461.063525,1787.414664,1116.454827,2053.885597
5,Nigeria Immigrants,278.411215,672.126784,45.169397,71.936914,595.939381,128.821711,20.586363
6,China Immigrants,4813.164293,488.513202,1562.157197,581.696769,2788.044092,1236.688423,10571.889163
7,India Immigrants,2900.345625,20830.876730,594.534855,465.109357,3580.074563,472.346273,2058.636296
8,Refugees,6447.571745,5062.007486,4331.569185,1012.077960,3732.993335,1751.975266,2822.707075
9,No certificate; diploma or degree,5849.930315,7160.087461,6307.876956,4673.418814,4755.007182,2928.546891,4340.555452


In [282]:
cities_crime_dfs = [pd.read_csv(f"CrimeData/{city}_Crime_Data.csv") for city in cities]

for df in cities_crime_dfs:
    df["GEO"] = df["GEO"].str.replace(r"\s\[\d+(?:,\d+)*\]", '', regex=True)
    df["Violations"] = df["Violations"].str.replace(r"\s\[\d+(?:,\d+)*\]", '', regex=True)

cleaned_cities_cime_dfs = [df[df["Statistics"] == "Rate per 100,000 population"].loc[:, ["REF_DATE", "Violations", "Statistics", "VALUE"]] for df in cities_crime_dfs]

cleaned_cities_cime_dfs[0].head(10)

for index in range(len(cities)):
    cleaned_cities_cime_dfs[index].rename(columns={"VALUE": cities[index]}, inplace=True)

In [283]:
merged_cities_crime_df = cleaned_cities_cime_dfs[0]

for df in cleaned_cities_cime_dfs[1:]:
    merged_cities_crime_df = pd.merge(merged_cities_crime_df, df.drop(columns=["Violations", "Statistics"]), how="left", on="REF_DATE")

merged_cities_crime_df.reset_index(inplace=True)
merged_cities_crime_df.drop(columns="index",  inplace=True)
merged_cities_crime_df.set_index("REF_DATE", inplace=True)
merged_cities_crime_df

,Violations,Statistics,Toronto,Brampton,Montreal,Halifax,Calgary,Fredericton,Vancouver
REF_DATE,,,,,,,,,
2015,"Total, all violations","Rate per 100,000 population",3818.31,3059.30,4947.02,5501.50,5431.29,6308.71,8264.09
2016,"Total, all violations","Rate per 100,000 population",3938.59,3100.57,4727.04,5337.91,5670.31,6534.47,8776.22
2017,"Total, all violations","Rate per 100,000 population",4142.40,3221.90,4709.81,5691.71,5860.16,6625.72,8719.18
2018,"Total, all violations","Rate per 100,000 population",4529.21,3335.07,4534.26,5773.81,6528.90,6009.45,8729.38
2019,"Total, all violations","Rate per 100,000 population",4589.50,2912.61,4836.96,5349.86,7050.56,6446.83,8754.07
2020,"Total, all violations","Rate per 100,000 population",3865.85,2540.33,4311.27,5059.29,5878.82,6536.24,7428.96
2021,"Total, all violations","Rate per 100,000 population",3991.92,2507.39,4608.61,5251.49,5705.70,6927.35,6426.91
2022,"Total, all violations","Rate per 100,000 population",4601.99,3055.66,5163.13,6029.66,6126.09,8706.11,6609.66
2023,"Total, all violations","Rate per 100,000 population",5326.74,3408.14,5415.87,5967.30,5681.60,8852.26,6924.62


In [ ]:
# Plot 2015 crime rate with the cleaned census data